In [3]:
import pandas as pd
import joblib
import os
from deep_translator import GoogleTranslator # Importamos el traductor

print("🔄 Cargando IA y activando traductor...")

# 1. CARGAR MODELO
ruta_modelo = '../models/modelo_spam_entrenado.pkl'
if os.path.exists(ruta_modelo):
    model = joblib.load(ruta_modelo)
else:
    print("❌ Error: No encuentro el modelo.")

# 2. RECUPERAR VOCABULARIO (Palabras que la IA conoce)
ruta_datos = r"c:\Users\eriko\Spam_Classifier_Project\data\raw\email-spam-classification-dataset-csv\emails.csv"
df_ref = pd.read_csv(ruta_datos, nrows=1)
columnas_palabras = df_ref.drop(columns=['Email No.', 'Prediction']).columns

# 3. FUNCIÓN DE PREDICCIÓN CON TRADUCCIÓN
def analizar_correo_espanol(mensaje_espanol):
    try:
        # A. TRADUCIR (Español -> Inglés)
        traductor = GoogleTranslator(source='es', target='en')
        mensaje_ingles = traductor.translate(mensaje_espanol)
        
        print(f"   (Traducido internamente a: '{mensaje_ingles}')")
        
        # B. CONVERTIR A NÚMEROS
        datos_entrada = pd.DataFrame(0, index=[0], columns=columnas_palabras)
        palabras = mensaje_ingles.lower().split()
        
        encontradas = 0
        for palabra in palabras:
            if palabra in datos_entrada.columns:
                datos_entrada.loc[0, palabra] += 1
                encontradas += 1
        
        print(f"   (La IA reconoció {encontradas} palabras clave)")

        # C. PREDECIR
        prediccion = model.predict(datos_entrada)[0]
        probabilidad = model.predict_proba(datos_entrada)[0][1]
        
        return prediccion, probabilidad
        
    except Exception as e:
        print(f"⚠️ Error en la traducción: {e}")
        return 0, 0

# 4. ¡PRUEBA EN ESPAÑOL!
print("\n--- 🕵️ DETECTOR DE SPAM (ESPAÑOL) ---")
print("Escribe 'salir' para terminar.")

while True:
    print("\n------------------------------------------------")
    texto = input("✍️ Escribe un correo en ESPAÑOL: ")
    
    if texto.lower() == 'salir':
        break
        
    resultado, certeza = analizar_correo_espanol(texto)
    
    if resultado == 1:
        print(f"🛑 ¡CUIDADO! Es SPAM. (Probabilidad: {certeza*100:.1f}%)")
    else:
        print(f"✅ Es correo SEGURO. (Probabilidad de Spam: {certeza*100:.1f}%)")

🔄 Cargando IA y activando traductor...

--- 🕵️ DETECTOR DE SPAM (ESPAÑOL) ---
Escribe 'salir' para terminar.

------------------------------------------------
   (Traducido internamente a: 'free money for you')
   (La IA reconoció 4 palabras clave)
🛑 ¡CUIDADO! Es SPAM. (Probabilidad: 86.0%)

------------------------------------------------
   (Traducido internamente a: 'click here to earn free money')
   (La IA reconoció 5 palabras clave)
🛑 ¡CUIDADO! Es SPAM. (Probabilidad: 94.2%)

------------------------------------------------
   (Traducido internamente a: 'accept this end of year offer')
   (La IA reconoció 6 palabras clave)
🛑 ¡CUIDADO! Es SPAM. (Probabilidad: 88.9%)

------------------------------------------------
